# TD0 — Limites OLTP et transition DWH/OLAP

## Objectifs

- Diagnostiquer limites OLTP pour analyses.
- Comprendre nécessité DWH/OLAP.
- Planifier transition.

## Diagnostic OLTP

- **Requêtes typiques** : Stock par produit/magasin (SELECT simple).
- **Limites** : Pas d'analyses temporelles/multi-dim (ex. CA par mois/catégorie).
- **Performance** : Normalisation ⇒ jointures lentes pour agrégations.

## Simulation limite OLTP (exemple Python simple)

Imaginons une base OLTP normalisée avec tables Produit, Magasin, Vente.

In [ ]:
# Simulation OLTP : requêtes lentes pour analyse
# (Pas de vraie DB, juste illustration)
print("Exemple requête OLTP : stock produit P01")
print("SELECT quantite FROM stock WHERE produit_id = 'P01' AND magasin_id = 'M01';")
print("Limite : Pour CA total par mois, jointures coûteuses.")
print("Solution : DWH avec modèle étoile pour agrégations rapides.")

## Schéma cible DWH

Transition vers modèle étoile (TD1).

## Plan transition

1. Audit sources OLTP.
2. Modélisation DWH.
3. ETL initial.